In [ ]:
import numpy as np 
import pandas as pd

import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.image import random_crop
from PIL import Image
import cv2
import random
import matplotlib.pyplot as plt

In [ ]:
# Define datasets and corresponding folders
datasets = {
    "refuge": {
        "image_folder": "/kaggle/input/glaucoma-datasets/REFUGE/test/Images",
        "mask_folder": "/kaggle/input/glaucoma-datasets/REFUGE/test/Masks",
        "output_image_folder": "/kaggle/working/refuge_images",
        "output_mask_folder": "/kaggle/working/refuge_masks"
    },
    "origa": {
        "image_folder": "/kaggle/input/glaucoma-datasets/ORIGA/Images_Square",
        "mask_folder": "/kaggle/input/glaucoma-datasets/ORIGA/Masks_Square",
        "output_image_folder": "/kaggle/working/origa_images",
        "output_mask_folder": "/kaggle/working/origa_masks"
    },
    "g1020": {
        "image_folder": "/kaggle/input/glaucoma-datasets/G1020/Images_Square",
        "mask_folder": "/kaggle/input/glaucoma-datasets/G1020/Masks_Square",
        "output_image_folder": "/kaggle/working/g1020_images",
        "output_mask_folder": "/kaggle/working/g1020_masks"
    }
}

# Iterate over each dataset
for dataset_name, paths in datasets.items():
    image_folder = paths["image_folder"]
    mask_folder = paths["mask_folder"]
    output_image_folder = paths["output_image_folder"]
    output_mask_folder = paths["output_mask_folder"]

    # Create output directories if they do not exist
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_mask_folder, exist_ok=True)

    # Process all images in the image folder
    image_files = os.listdir(image_folder)

    for img_name in image_files:
        if img_name.endswith('.jpg') or img_name.endswith('.png'):  # Check for image file extensions
            img_path = os.path.join(image_folder, img_name)
            
            # Convert the image to grayscale
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                # Save the grayscale image as PNG
                grayscale_img_path = os.path.join(output_image_folder, img_name.replace('.jpg', '.png').replace('.png', '.png'))
                cv2.imwrite(grayscale_img_path, image)
            
            # Find the corresponding mask
            mask_name = img_name.replace('.jpg', '.png')
            mask_path = os.path.join(mask_folder, mask_name)

            # Process and invert the mask colors (black to white, everything else to black)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is not None:
                mask = np.where(mask == 0, 255, 0)  # Invert the colors: black -> white, everything else -> black
                
                # Save the processed mask as PNG
                processed_mask_path = os.path.join(output_mask_folder, mask_name)
                cv2.imwrite(processed_mask_path, mask)

    print(f"Processing and saving complete for dataset: {dataset_name}")

print("Processing and saving complete.")

In [ ]:
# Resize and normalize parameters
IMG_SIZE = (256, 256) 
BATCH_SIZE = 8        


# Define the function for loading and preprocessing 
def load_and_preprocess_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image_resized = cv2.resize(image, IMG_SIZE)
    # Normalize the image
    image_normalized = image_resized / 255.0  # Normalize to [0, 1]
    # Add channel dimension (256, 256, 1)
    return np.expand_dims(image_normalized, axis=-1)
    
def load_and_preprocess_mask(mask_path):
    # Load the mask in grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # Resize the mask
    mask_resized = cv2.resize(mask, IMG_SIZE)
    # Normalize the mask (binary mask)
    mask_normalized = mask_resized / 255.0  # Normalize to [0, 1]
    # Add channel dimension (256, 256, 1)
    return np.expand_dims(mask_normalized, axis=-1)
      

# Folders for the three pairs of images and masks
folders = {
    "refuge": {
        "image_folder": "/kaggle/working/refuge_images",
        "mask_folder": "/kaggle/working/refuge_masks"
    },
    "origa": {
        "image_folder": "/kaggle/working/origa_images",
        "mask_folder": "/kaggle/working/origa_masks"
    },
    "g1020": {
        "image_folder": "/kaggle/working/g1020_images",
        "mask_folder": "/kaggle/working/g1020_masks"
    }
}


# Iterate through each dataset
datasets = {}
for dataset_name, paths in folders.items():
    image_folder = paths["image_folder"]
    mask_folder = paths["mask_folder"]

    images = []
    masks = []

    # Iterate through image and mask files
    image_files = os.listdir(image_folder)
    for img_name in image_files:
        if img_name.endswith('.png'):  # Check for image file extensions
            img_path = os.path.join(image_folder, img_name)
            mask_name = img_name.replace('.png', '.png')  # Ensure mask name matches
            msk_path = os.path.join(mask_folder, mask_name)

            # Load and process the image and mask
            image = load_and_preprocess_image(img_path)
            mask = load_and_preprocess_mask(msk_path)

            # Append the processed image and mask
            images.append(image)
            masks.append(mask)

    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)

    #Convert the arrays to tensors
    image_tensor = tf.convert_to_tensor(images, dtype=tf.float32)
    mask_tensor = tf.convert_to_tensor(masks, dtype=tf.float32)
    # Create a TensorFlow dataset from the images and masks
    dataset = tf.data.Dataset.from_tensor_slices((image_tensor, mask_tensor))

    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE)

    # Store the dataset
    datasets[dataset_name] = dataset

# Now you can access the datasets like this:
refuge_dataset = datasets["refuge"]
origa_dataset = datasets["origa"]
g1020_dataset = datasets["g1020"]


print("Dataset created and ready for training.")

In [ ]:
#Load metrics and loss function as custom parameters
@tf.keras.utils.register_keras_serializable()
def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.where(tf.less(tf.range(tf.shape(y_true)[2]), tf.shape(y_true)[2] // 2), 2.0, 1.0)
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weighted_bce = bce * weights
    return tf.reduce_mean(weighted_bce)

@tf.keras.utils.register_keras_serializable()
def dice_coefficient(y_true, y_pred):
    # Cast both y_true and y_pred to float32 to ensure compatibility
    y_true_f = tf.keras.backend.flatten(tf.cast(y_true, tf.float32))
    y_pred_f = tf.keras.backend.flatten(tf.cast(y_pred, tf.float32))
    
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + 1e-6) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1e-6)
    
    return dice

@tf.keras.utils.register_keras_serializable()
def iou(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(tf.cast(y_true, tf.float32))
    y_pred_f = tf.keras.backend.flatten(tf.cast(y_pred, tf.float32))

    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    iou_metric = (intersection ) / (union )
    return iou_metric

In [ ]:
#Load the model
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/best_models/keras/default/1/Best Models/xnet.keras')

In [ ]:
#Evaluate on REFUGE
model.evaluate(refuge_dataset)

In [ ]:
#Evaluate on random 400 images on ORIGA
model.evaluate(origa_dataset.take(50))

In [ ]:
#Evaluate on random 400 images on G1020
model.evaluate(g1020_dataset.take(50))